In [ ]:
# install unsloth for finetuning
%%capture
import torch
major_version, minor_version = torch.cuda.get_device_capability()
# Must install separately since Colab has torch 2.2.1, which breaks packages
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
if major_version >= 8:
    # Use this for new GPUs like Ampere, Hopper GPUs (RTX 30xx, RTX 40xx, A100, H100, L40)
    !pip install --no-deps packaging ninja einops flash-attn xformers trl peft accelerate bitsandbytes
else:
    # Use this for older GPUs (V100, Tesla T4, RTX 20xx)
    !pip install --no-deps xformers trl peft accelerate bitsandbytes
pass

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Sun Mar 31 08:47:59 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off | 00000000:00:04.0 Off |                    0 |
| N/A   31C    P0              43W / 400W |      5MiB / 40960MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [ ]:
#login to huggingface
from google.colab import userdata
secret_hf = userdata.get('HUGGINGFACE_TOKEN')
!huggingface-cli login --token $secret_hf

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = False # Use 4bit quantization to reduce memory usage. Can be False.

#load base model + tokenizer
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "mistralai/Mistral-7B-Instruct-v0.2", # Choose ANY! eg teknium/OpenHermes-2.5-Mistral-7B
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

config.json:   0%|          | 0.00/596 [00:00<?, ?B/s]

==((====))==  Unsloth: Fast Mistral patching release 2024.3
   \\   /|    GPU: Tesla V100-SXM2-16GB. Max memory: 15.773 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.2.1+cu121. CUDA = 7.0. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. Xformers = 0.0.25.post1. FA = False.
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

In [ ]:
# settings for the tokenizer
tokenizer.padding_side = 'right'
tokenizer.pad_token = tokenizer.eos_token
tokenizer.add_eos_token = True
tokenizer.bos_token, tokenizer.eos_token

('<s>', '</s>')

In [ ]:
# set the hyperparemeters for the LoRA adapters
model = FastLanguageModel.get_peft_model(
    model,
    r = 64, # Adjust the size of the LoRA matrices (Suggested 8, 16, 32, 64, 128)
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16, # adjust the scaling factor for the weigths
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    use_gradient_checkpointing = True,
    random_state = 42,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2024.3 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [ ]:
# Load the dataset for training the model
from datasets import load_dataset
# Load the rationale augmented dataset for finetuning from Hugging Face
dataset = load_dataset("Koni99/ISOT_finetuning_final")

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
# inspect the prompt template
dataset["train"]["prompt"][:3]

['<s>[INST] You are an expert AI disinformation detector trained to detect fake news. Is this article real or fake news? Analyze the given text in detail as a fact checker would. Explain your reasoning step-by-step.  Check for misleading information, false claims, biased language. If the news article is real, respond with 1 for \'True\', if fake, respond with 0 for \'False\'.  Structure your answer as a JSON object with the two keys \'rationale\' for your analytic reasoning and \'prediction\' for your final prediction.  The values for \'prediction\' are binary. Either 1 for a real article or 0 for fake news. In \'rationale\' you provide a string with your detailed reasoning process that led you to the decision you took.   Remember if the article contains misleading information, false claims, biased language or any other hints of disinformation your prediction should be 0 for fake news, else 1 for true news.   To summarize the JSON object looks like this: \'rationale\': <your_analytic_r

#Training

In [ ]:
# setting for the Supervised-fine-tuning trainer
from trl import SFTTrainer
from transformers import TrainingArguments

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset["train"],
    dataset_text_field = "prompt", # important: pick correct column of the dataset with the input text in the right format for the SLM
    max_seq_length = max_seq_length,
    #dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 8,
        gradient_accumulation_steps = 2,
        warmup_steps = 5,
        num_train_epochs=1,
        #max_steps = None,
        learning_rate = 2e-4,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 42,
        output_dir = "outputs",
    ),
)

Map:   0%|          | 0/41178 [00:00<?, ? examples/s]

In [ ]:
# calculates the number of trained parameters
trainable, total = model.get_nb_trainable_parameters()
print(f"Trainable: {trainable} | total: {total} | Percentage: {trainable/total*100:.4f}%")

Trainable: 167772160 | total: 7409504256 | Percentage: 2.2643%


In [ ]:
#@title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = NVIDIA A100-SXM4-40GB. Max memory = 39.564 GB.
14.674 GB of memory reserved.


In [ ]:
# start the training run
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 41,178 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 8 | Gradient Accumulation steps = 2
\        /    Total batch size = 16 | Total steps = 2,574
 "-____-"     Number of trainable parameters = 167,772,160


Step,Training Loss
1,2.336100
2,2.248400
3,2.137400
4,2.117600
5,1.764100
6,1.689900
7,1.680600
8,1.678400
9,1.429700
10,1.499000


In [ ]:
#@title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

15586.6669 seconds used for training.
259.78 minutes used for training.
Peak reserved memory = 28.564 GB.
Peak reserved memory for training = 13.89 GB.
Peak reserved memory % of max memory = 72.197 %.
Peak reserved memory for training % of max memory = 35.108 %.


In [ ]:
# save the LoRA adapter weights: Important this is not a complete model only the LoRA weights
new_model = "mistral-7b-RAFT-unsloth2"
model.save_pretrained(new_model, use_temp_dir=False) # Local saving
model.push_to_hub(new_model, use_temp_dir=False) # Online saving

README.md:   0%|          | 0.00/591 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/671M [00:00<?, ?B/s]

Saved model to https://huggingface.co/mistral-7b-RAFT-unsloth2


In [ ]:
# inspect the model structure
model.config.use_cache = True
model.eval()


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): MistralForCausalLM(
      (model): MistralModel(
        (embed_tokens): Embedding(32000, 4096)
        (layers): ModuleList(
          (0-31): 32 x MistralDecoderLayer(
            (self_attn): MistralSdpaAttention(
              (q_proj): lora.Linear(
                (base_layer): Linear(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=64, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=64, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (k_proj): lora.Linear(
                (base_layer): Linear(in_features=4096, out_fe

In [ ]:
#load fine-tuned adapter as new_model
# Load model directly
from transformers import AutoModelForCausalLM, AutoTokenizer
base_model_name = "mistralai/Mistral-7B-Instruct-v0.2"
new_model_name = "mistral-7b-RAFT-unsloth2-merged"

In [ ]:
# reload base model
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map={"": 0},
)

config.json:   0%|          | 0.00/596 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

In [ ]:
# Reload tokenizer
tokenizer = AutoTokenizer.from_pretrained(base_model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

tokenizer_config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel


adapter_model_name = "Koni99/mistral-7b-RAFT-unsloth2"

# Load the base model
#base_model = AutoModelForCausalLM.from_pretrained(base_model_name)

# Load the adapter model
adapter_model = PeftModel.from_pretrained(base_model, adapter_model_name)

# Merge the adapter into the base model
merged_model = adapter_model.merge_and_unload()

adapter_config.json:   0%|          | 0.00/739 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/671M [00:00<?, ?B/s]

In [ ]:
# Save the merged model: This is a complete SLM now, including the LoRA adapters
merged_model.save_pretrained(new_model_name,safe_serialization=True)
tokenizer.save_pretrained(new_model_name)

('mistral-7b-RAFT-unsloth2-merged/tokenizer_config.json',
 'mistral-7b-RAFT-unsloth2-merged/special_tokens_map.json',
 'mistral-7b-RAFT-unsloth2-merged/tokenizer.model',
 'mistral-7b-RAFT-unsloth2-merged/added_tokens.json',
 'mistral-7b-RAFT-unsloth2-merged/tokenizer.json')

In [ ]:
# Push the model and tokenizer to the Hugging Face Model Hub
merged_model.push_to_hub(new_model_name, use_temp_dir=False)
tokenizer.push_to_hub(new_model_name, use_temp_dir=False)

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.18k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Koni99/mistral-7b-RAFT-unsloth2-merged/commit/488fe8fe5e516a6e8bbbc7aa6b68f38f4d79369a', commit_message='Upload tokenizer', commit_description='', oid='488fe8fe5e516a6e8bbbc7aa6b68f38f4d79369a', pr_url=None, pr_revision=None, pr_num=None)

In [1]:
#additionally the model can be saved model in a gguf format, which compresses the size and makes it easier to run it locally
# Save to 8bit Q8_0
if False: model.save_pretrained_gguf("/home/mistral-7b-RAFT-unsloth-gguf-8bit", tokenizer,)
if False: model.push_to_hub_gguf("mistral-7b-RAFT-unsloth-gguf-8bit", tokenizer, token = "", use_temp_dir = True)

# Save to 16bit GGUF
if True: model.save_pretrained_gguf("/home/mistral-7b-RAFT-unsloth2-gguf-16bit", tokenizer, quantization_method = "f16")
if False: model.push_to_hub_gguf("mistral-7b-RAFT-unsloth-gguf-16bit", tokenizer, quantization_method = "f16", token = "",use_temp_dir = True)

NameError: name 'model' is not defined